In [ ]:
%pip install auto-sklearn

     |████████████████████████████████| 6.4 MB 6.6 MB/s 
     |████████████████████████████████| 38.1 MB 5.4 MB/s 
     |████████████████████████████████| 22.3 MB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.2 MB/s 
     |████████████████████████████████| 837 kB 8.8 MB/s 
     |████████████████████████████████| 4.3 MB 34.3 MB/s 
     |████████████████████████████████| 4.0 MB 33.9 MB/s 
     |████████████████████████████████| 260 kB 39.6 MB/s 
     |████████████████████████████████| 136 kB 33.1 MB/s 
     |████████████████████████████████| 596 kB 40.0 MB/s 
     |████████████████████████████████| 45 kB 3.6 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.7-py3-none-any.whl size=6602873 sha256=55a1667618b84dff7ec5e3c11a0776726210ff86934ee1063adbf0d03fdd512e
  Stored in directory: /root/.cache/pip/wheels/ba/43/5c/2fbe6fd19e3af314cbc4aa808378068d8ddd6792064f4a2448
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7043 sha256=f4d3b4

In [ ]:
import sklearn.datasets
import sklearn.model_selection
import autosklearn.regression
import autosklearn.metrics
import pandas as pd

In [ ]:
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id = name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  return (X_train, X_test, y_train, y_test)

In [ ]:
X_train, X_test, y_train, y_test = dataSetup(223)
X_train.head()

,company1,company2,company3,company4,company5,company6,company7,company8,company9
327,149,60,49,22,140,67,38,44,49
628,332,164,55,38,264,107,149,65,8
456,218,34,1,92,183,95,135,61,8
196,101,57,45,0,59,43,68,47,51
697,376,207,61,107,242,116,8,70,104


In [ ]:
estimator_askl = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=3600,seed=42,resampling_strategy='cv',)
estimator_askl.fit(X_train, y_train, dataset_name='stock')
prediction = estimator_askl.predict(X_test)

[WARNING] [2022-05-04 18:26:54,949:Client-AutoMLSMBO(42)::stock] Dataset stock already in meta-data. Removing occurence.


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py:691: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,


In [ ]:
(prediction == y_test).sum()/len(prediction)

0.0

In [ ]:
print(estimator_askl.sprint_statistics())

auto-sklearn results:
  Dataset name: stock
  Metric: r2
  Best validation score: 0.675141
  Number of target algorithm runs: 122
  Number of successful target algorithm runs: 118
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 4
  Number of target algorithms that exceeded the memory limit: 0



In [ ]:
from pprint import pprint
pprint(estimator_askl.show_models(), indent=4)

{   21: {   'cost': 0.4133855466285311,
            'ensemble_weight': 0.06,
            'estimators': [   {   'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fdbc05666d0>,
                                  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fdbbfb2b990>,
                                  'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7fdbc0020a50>,
                                  'sklearn_regressor': GaussianProcessRegressor(alpha=0.00025503968621419157,
                         kernel=RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1, 1]),
                         n_restarts_optimizer=10, normalize_y=True,
                         random_state=42)},
                              {   'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fdbbfaf9cd0>,
        

In [ ]:
for run_key in estimator_askl.automl_.runhistory_.data:
    print('#########')
    print(run_key)
    print(estimator_askl.automl_.runhistory_.data[run_key])

#########
RunKey(config_id=1, instance_id='{"task_id": "stock"}', seed=0, budget=0.0)
RunValue(cost=0.33863067676584346, time=12.513041257858276, status=<StatusType.SUCCESS: 1>, starttime=1651688815.1411679, endtime=1651688827.6765707, additional_info={'duration': 12.154729127883911, 'num_run': 2, 'train_loss': 0.04576279180687318, 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=2, instance_id='{"task_id": "stock"}', seed=0, budget=0.0)
RunValue(cost=0.5618191233708396, time=261.3158001899719, status=<StatusType.SUCCESS: 1>, starttime=1651688827.7271795, endtime=1651689089.0582566, additional_info={'duration': 261.2185580730438, 'num_run': 3, 'train_loss': 0.0102851949426582, 'configuration_origin': 'Initial design'})
#########
RunKey(config_id=3, instance_id='{"task_id": "stock"}', seed=0, budget=0.0)
RunValue(cost=0.4281620198731926, time=136.00591707229614, status=<StatusType.SUCCESS: 1>, starttime=1651689089.1627257, endtime=1651689225.183232, additional_info=

In [ ]:
print("Accuracy score", sklearn.metrics.r2_score(y_test, prediction))


Accuracy score 0.6942807641471165


In [ ]:
print(estimator_askl.leaderboard())

          rank  ensemble_weight                 type      cost   duration
model_id                                                                 
40           1             0.02        random_forest  0.324859   8.057242
54           2             0.18        random_forest  0.327541   7.687591
45           3             0.22        random_forest  0.327568   8.407757
31           4             0.06  k_nearest_neighbors  0.341721   2.480791
35           5             0.14  k_nearest_neighbors  0.346000   2.591665
91           6             0.02  k_nearest_neighbors  0.368101   2.806118
62           7             0.08                  sgd  0.378492   5.011304
93           8             0.06        liblinear_svr  0.384734   2.586537
68           9             0.04        decision_tree  0.408264   2.464861
21          10             0.06     gaussian_process  0.413386  16.960043
24          11             0.12     gaussian_process  0.413911  15.450598


In [ ]:
print("Accuracy score", sklearn.metrics.mean_squared_error(y_test, prediction))

Accuracy score 871.7666371681378
